<div align="center"; span style="color:#336699"><b><h2>Minicurso pyForTraCC </h2></b></div>
<div align="center"; span style="color:#336699"><b><h3>1. Exemplo introdutório</h3></b></div>
<hr style="border:2px solid #0077b9;">
<br/>
<div style="text-align: center;font-size: 90%;">
   <sup>Autor: <a href="https://www.linkedin.com/in/helvecio-leal/"> Helvécio B. Leal Neto, <i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup><t>&nbsp;</t> 
   <br/><br/>
    National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:fortracc.project@inpe.br">fortracc.project@inpe.br</a>
    <br/><br/>
    Last Update: Sep 6, 2025
</div>

<br/>

<div style="text-align: center; margin: 0 auto; width: 80ch;">
<b>Resumo.</b> Este notebook faz parte do minicurso de introdução ao <a href="https://github.com/fortracc/pyfortracc">pyfortracc</a> e apresenta um exemplo básico de rastreamento de dados sintéticos</a>.
</div>

<div align="center">

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fortracc/pyfortracc/blob/main/examples/WORCAP-Minicourse/1_Basic_Tracking/1_Basic_Tracking.ipynb) 

</div>
<br/>

### Roteiro
 [1. Instalação](#install)<br>
 [2. Dados de Entrada](#input)<br>
 [3. Função de Leitura](#data)<br>
 [4. Parâmetros (Name_list)](#namelist)<br>
 [5. Rotina de Rastreamento](#track)<br>
 [6. Tabela de Rastreamento](#tracktable)<br>
 [7. Visualização de Rastreamento](#visualization)<br>


##### 1. Instalação <a class="anchor" id="install"></a>
Para instalar a biblioteca pyFortraCC, você pode usar o pip. Execute o seguinte comando

In [ ]:
# Comando para instalação da ultima versão estável do pyfortracc
!python -m pip install -qqq -U pyfortracc > /dev/null 2>&1 && echo "✅ pyfortracc instalado com sucesso!" || echo "❌ Erro na instalação"

Após a instalação caso tudo ocorra bem você verá a mensagem "✅ pyfortracc instalado com sucesso!". 

Em seguida, você pode importar a biblioteca a biblioteca e verificar a versão instalada.

In [ ]:
# reload notebook
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../../")  # Adiciona o diretório pai ao sys.path

In [ ]:
import pyfortracc
print('pyFortracc version', pyfortracc.__version__)

##### 2. Dados de Entrada <a class="anchor" id="input"></a>

Para este exemplo, usaremos dados sintéticos gerados por uma função interna do pyfortracc chamada `bubble_simulation()`. Esta função cria um conjunto de dados fictícios que simulam a movimentação de objetos ao longo do tempo e armazena esses dados como imagens no formato png no diretório especificado.

In [ ]:
pyfortracc.utilities.bubble_simulation(dir='input/')

Caso tudo ocorra bem, você poderá explorar o conteúdo das simulações geradas dentro do diretório especificado. Neste caso, o diretório é `.input/`. Você pode navegar até esse diretório para visualizar os arquivos de imagem gerados pela simulação. Cada arquivo de imagem representa um instante no tempo da simulação, mostrando a posição dos objetos simulados naquele momento específico. Com o comando abaixo, você pode listar os arquivos presentes no diretório `.input/` para verificar se as imagens foram criadas corretamente.

In [ ]:
!ls -s input/

Na célula abaixo, você pode visualizar uma das imagens geradas pela simulação. A imagem `frame_00.png` representa o estado inicial da simulação, mostrando a posição dos objetos no primeiro instante de tempo.

<img src="input/frame_00.png" alt="Descrição da imagem" width="10%">

##### 3. Função de Leitura <a class="anchor" id="data"></a>
A função `read_data()` é responsável por ler os dados de entrada, que neste caso são imagens PNG. Ela utiliza a biblioteca PIL (Python Imaging Library) para abrir e converter as imagens em arrays binários.  
Essa função retorna um array numpy contendo os dados lidos de cada imagem.  
Para `pyfortracc`, cada imagem é tratada como um "frame" ou "time step" na sequência temporal dos dados, e a saída da função deve ser um array numpy 2D.

A célula abaixo demonstra como deve ser definida a função `read_data()` para ler imagens PNG.

In [ ]:
from PIL import Image
import numpy as np

def read_function(path):
    # Abre a imagem e converte para escala de cinza
    img = Image.open(path).convert("L")
    # Converte a imagem em um array NumPy
    arr = np.array(img)
    # Binariza a imagem: bolhas (1) e fundo (0)
    arr = np.where(arr < 250, 1, 0)
    return arr

Para verificar se a função `read_data()` está funcionando corretamente, você pode tentar abrir uma das imagens geradas pela simulação.  A célula abaixo tenta abrir a imagem `input/frame_00.png` <br>e exibir seu conteúdo como um array numpy. Se a função estiver correta, você verá a matriz de valores binários representando a imagem.

In [ ]:
read_function('input/frame_00.png')

Caso tudo ocorra bem, você verá a matriz de valores binários com dimensões de (50, 50), indicando que a imagem foi lida corretamente.<br>
O pyfortracc também conta com uma função interna chamada `plot_animation()` que permite visualizar a sequência temporal dos dados lidos.<br>
A célula abaixo demonstra como usar essa função para criar uma animação dos frames lidos pela função `read_data()`.<br>
Como argumentos da função, você deve passar o caminho dos arquivos de entrada `input/*.png` e a função de leitura `read_function`.

In [ ]:
pyfortracc.plot_animation(path_files='input/*.png', 
                          read_function=read_function)

##### 4. Parâmetros (Name_list) <a class="anchor" id="namelist"></a>
O dicionário `name_list` é uma estrutura de dados que armazena os parâmetros necessários para o processo de rastreamento no pyfortracc. <br>
Cada chave do dicionário representa um parâmetro específico, e o valor associado a essa chave define o comportamento do rastreamento.<br> Você pode encontrar uma descrição detalhada de cada parâmetro na [documentação oficial do pyfortracc](https://fortracc.github.io/pyfortracc/).

In [ ]:
name_list = {}
name_list['input_path'] = 'input/' # Caminho para os arquivos de entrada
name_list['output_path'] = 'output/' # Caminho para os arquivos de saída
name_list['thresholds'] = [1] # Lista de limiares de intensidade a serem usados no processo de segmentação
name_list['min_cluster_size'] = [3] # Lista que contém o tamanho mínimo dos clusters
name_list['operator'] = '>=' # '>= - <=' ou '=='
name_list['timestamp_pattern'] = 'frame_%M.png' # Padrão de nome de arquivo de timestamp
name_list['delta_time'] = 1 # em minutos

In [ ]:
pyfortracc.track(name_list, read_function)

In [ ]:
import pandas as pd
import glob

tracking_files = sorted(glob.glob(name_list['output_path'] + '/track/trackingtable/*.parquet'))
tracking_table = pd.concat(pd.read_parquet(f) for f in tracking_files)
# tracking_table.head()

In [ ]:
pyfortracc.plot(name_list=name_list, 
                read_function=read_function,
                timestamp='1900-01-01 00:19:00',
                )

In [ ]:
pyfortracc.plot_animation(name_list= name_list, read_function=read_function,
                          start_timestamp= tracking_table['timestamp'].min().strftime('%Y-%m-%d %H:%M:%S'), info=True, info_cols=['uid','lifetime', 'status'],
                          end_timestamp= tracking_table['timestamp'].max().strftime('%Y-%m-%d %H:%M:%S'))